# Lab 3 - Computaional Intelligence 2024

In [1]:
from collections import namedtuple, deque
from queue import *
from random import *
from tqdm.auto import tqdm
import numpy as np

c:\Users\Lorenzo Ugoccioni\Documents\PoliTo\LM - III anno\Computational Intelligence\Labs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 100_000

action = namedtuple('Action', ('pos1', 'pos2'))

In [71]:
#   Create the game table
#   state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))


#   Default game: search for number ordered increasing
default_game = True 

#   If we wanna search a Specific configuration, ENTER here the configuration following the rules:
#   -   MUST contain ONLY one "0";
#   -   write number from 1 to PUZZLE_DIM**2 - 1 in a list

GOAL_STATE = [1, 2, 3, 0]

if not default_game:
    if GOAL_STATE.count(0) != 1:
        print("You insered too many 0s in your goal state!\n", GOAL_STATE)
        exit(1)

    if len(set(GOAL_STATE)) != PUZZLE_DIM**2:
        print("You haven't created a feasible solution!\n", GOAL_STATE)
        exit(1)

    GOAL_STATE = np.array(GOAL_STATE).reshape((PUZZLE_DIM, PUZZLE_DIM))
else:

    GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

GOAL_STATE

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

In [72]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [73]:
#   Initial state of the game   #

INITIAL_STATE = [i for i in range(1, PUZZLE_DIM**2)] + [0]
shuffle(INITIAL_STATE)
INITIAL_STATE = np.array(INITIAL_STATE).reshape((PUZZLE_DIM, PUZZLE_DIM))
print(INITIAL_STATE)

[[1 7 3]
 [4 2 5]
 [6 0 8]]


In [56]:
def solution_check(state: np.ndarray) -> bool:
    return np.array_equal(state, GOAL_STATE)

In [60]:
def avoid_loop(state: np.ndarray, visited_state: list[np.ndarray]) -> bool:
    return any(np.array_equal(state, s) for s in visited_state)

### Randominzed Solution

In [74]:
state = INITIAL_STATE

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #print(state)
    state = do_action(state, choice(available_actions(state)))
    if solution_check(state):
        break
print(state, r+1)


Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 62471.81it/s]

[[5 0 7]
 [8 2 6]
 [1 4 3]] 100000


## PATH SEARCH STRATEGY

Mi serve tener conto di:
- Costo --> Quanti nodi ho esplorato -> nodi = azioni fatte per cambiare lo stato del gioco
- Qualità --> Minor numero di nodi necessari per arrivare alla soluzione -> nodi = azioni

OBIETTIVO --> ___Qualità vs Costo___

### Depth-First Strategy

Esploro in profondità ciascun nodo fino alla foglia, poi passo al figlio successivo.<br>
Criticità -> Soluzioni :
- ogni stato ha una mossa che mi fa tornare allo stato da cui sono venuto.<-- __CICLO__<br>
_SOL_: evitare di analizzare questa mossa -> __Pruning__ per ridurre lo spazio da esaminare

In [78]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()

explored_state.append(frontier.pop())
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    
    while not solution_check(state):
        actions = available_actions(state)

        for a in actions:
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.append(new_state)

        #print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 0it [00:00, ?it/s]

Depth-First: 11081it [08:58, 20.56it/s]


KeyboardInterrupt: 

### Breadth-First Strategy

Esploro prima tutti i figli di nodo allo stesso livello, poi passo ai figli, fino alle foglie

In [ ]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()

explored_state.append(frontier.pop())
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    
    while not solution_check(state):
        actions = available_actions(state)

        for a in actions:
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.appendleft(new_state)

        #print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 81960it [00:01, 67108.64it/s]


KeyboardInterrupt: 

### A* Strategy

In [ ]:
frontier = PriorityQueue()

special_set = compute_special_set()

state = special_set      # ({taken}, {not_taken})
frontier.put((f(state), state))       # [ ( {f=g+h}, { {taken}, {not_taken} } ), ]

counter = 0
_ , current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for tiles in current_state[1]:
            new_state = State(current_state.taken ^ {tiles}, current_state.not_taken ^ {tiles})     # "^" == XOR
            frontier.put((f(new_state), new_state))
        _ ,current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

# Show Solution

In [45]:
print(state,"\n", GOAL_STATE)

[[1 2]
 [3 0]] 
 [[1 2]
 [3 0]]
